In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
from typing import Callable
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from numba import njit
from scipy.stats import beta as beta_dist, norm
from scipy.optimize import brentq
from IPython.display import Image, Markdown, display

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 6), 'figure.dpi': 130,
                     'axes.titlesize': 'x-large', 'axes.labelsize': 'large',
                     'xtick.labelsize': 'medium', 'ytick.labelsize': 'medium'})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-info'>📝 {textwrap.fill(msg, width=100)}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized.")

# Part 3: Dynamic Models
## Chapter 3.04: Optimal Stopping Problems

### Table of Contents
1.  [The Structure of Optimal Stopping Problems](#1.-The-Structure-of-Optimal-Stopping-Problems)
    *   [1.1 The Secretary Problem: A Finite Horizon Example](#1.1-The-Secretary-Problem:-A-Finite-Horizon-Example)
    *   [1.2 The Bellman Equation for Optimal Stopping](#1.2-The-Bellman-Equation-for-Optimal-Stopping)
2.  [Application 1: The McCall Job Search Model](#2.-Application-1:-The-McCall-Job-Search-Model)
3.  [Continuous Time: The HJB Equation and Real Options](#3.-Continuous-Time:-The-HJB-Equation-and-Real-Options)
    *   [3.1 From Bellman to HJB](#3.1-From-Bellman-to-HJB)
    *   [3.2 Application: Irreversible Investment under Uncertainty](#3.2-Application:-Irreversible-Investment-under-Uncertainty)
4.  [Application 2: American Option Pricing](#4.-Application-2:-American-Option-Pricing)
5.  [Chapter Summary](#5.-Chapter-Summary)
6.  [Exercises](#6.-Exercises)

### Introduction: The Economics of When to Act
Many economic decisions are not about *how much* to do something, but *when* to do it. A firm decides when to make an irreversible investment. A job seeker decides when to accept a wage offer. A household decides when to sell their house. These are all **optimal stopping problems**.

An optimal stopping problem is a special type of dynamic programming problem where, at each point in time, the agent faces a simple binary choice: **STOP** or **CONTINUE**.
- **STOP:** End the process and receive a terminal payoff. This decision is irreversible.
- **CONTINUE:** Forgo the stopping payoff, receive any flow payoff for the current period, and face the same decision again in the next period, where the state of the world may have changed.

The agent must find a rule—a mapping from states to actions—that maximizes their total expected discounted payoff. This notebook introduces the formal structure of these problems and solves canonical applications from labor economics, corporate finance, and asset pricing.

### 1. The Structure of Optimal Stopping Problems

#### 1.1 The Secretary Problem: A Finite Horizon Example
The classic "Secretary Problem" provides a clean, analytical introduction to the logic of optimal stopping. An administrator wishes to hire the best secretary out of $N$ candidates, interviewed in random order. After each interview, the administrator must immediately decide to either hire the candidate (and stop the search) or reject them forever. The goal is to maximize the probability of hiring the single best candidate.

The state is the information available: the number of candidates interviewed so far ($k$) and whether the current candidate is the best seen *so far*. The optimal strategy can be found via backward induction. It takes the form of a simple threshold rule: reject the first $r-1$ candidates, and then hire the first subsequent candidate who is better than all previous ones. The optimal threshold $r$ for large $N$ approaches $N/e$, where $e \approx 2.718$. This means it is optimal to let the first $\approx 37\%$ of candidates go by to establish a benchmark, and then hire the next "best so far" candidate.

#### 1.2 The Bellman Equation for Optimal Stopping
For an infinite horizon problem, the Bellman equation for an optimal stopping problem has a simple and elegant form. The value of being in state `s` is the maximum of the value of stopping and the value of continuing.
$$ V(s) = \max \{ \text{StopValue}(s), \text{ContinueValue}(s) \} $$ 
Where the `ContinueValue(s)` is the flow payoff (if any) plus the discounted expected value of facing the problem again in the next period: 
$$ \text{ContinueValue}(s) = \text{FlowPayoff}(s) + \beta E[V(s')]$$
This structure lends itself perfectly to solution by value function iteration. The set of states where the agent is indifferent forms the **optimal stopping boundary**.

### 2. Application 1: The McCall Job Search Model
In the McCall model, an unemployed agent receives a benefit, `c`, and a wage offer, `w`, drawn from a distribution `p(w)`. The Bellman equation for an agent who has just received an offer `w` is:
$$ V(w) = \max \left\{ \underbrace{\frac{w}{1-\beta}}_{\text{StopValue}}, \quad \underbrace{c + \beta \int V(w') p(w') dw'}_{\text{ContinueValue}} \right\} $$

In [ ]:
sec("Solving for the McCall Reservation Wage")
BETA, C = 0.96, 25
WAGE_A, WAGE_B, WAGE_SCALE = 2, 5, 60
w_dist = beta_dist(a=WAGE_A, b=WAGE_B, scale=WAGE_SCALE)
w_grid = np.linspace(0.01, WAGE_SCALE, 150)
pdf_grid = w_dist.pdf(w_grid); pdf_grid /= np.sum(pdf_grid)

def reservation_wage_objective(w_bar):
    integrand = np.maximum(w_grid, w_bar) * pdf_grid
    integral = np.sum(integrand)
    v_continue = C + BETA * integral / (1 - BETA)
    v_stop = w_bar / (1 - BETA)
    return v_stop - v_continue

w_bar = brentq(reservation_wage_objective, a=0, b=WAGE_SCALE)
note(f"The reservation wage is w_bar = ${w_bar:.2f}")

fig, ax = plt.subplots()
ax.plot(w_grid, w_grid / (1 - BETA), lw=2.5, label='Value of Accepting Offer w')
ax.axhline((1 - BETA) * w_bar, lw=2.5, color='orange', label='Value of Continuing Search')
ax.axvline(w_bar, color='k', ls='--', label=f'Reservation Wage w* = {w_bar:.2f}')
ax.set(xlabel='Wage Offer (w)', ylabel='Lifetime Value', title='Optimal Stopping Policy in Job Search'); ax.legend()
plt.show()

### 3. Continuous Time: The HJB Equation and Real Options

#### 3.1 From Bellman to HJB
As the time step $\Delta t$ in a discrete model goes to zero, the Bellman equation converges to a differential equation known as the **Hamilton-Jacobi-Bellman (HJB) equation**. For an optimal stopping problem, the HJB equation partitions the state space into two regions:
1.  A **continuation region**, where the value of waiting is positive. Here, the value function $V(s)$ must satisfy a partial differential equation.
2.  A **stopping region**, where it is optimal to stop. Here, $V(s)$ is equal to the terminal payoff.

The **free boundary** is the border between these two regions, representing the optimal stopping boundary.

#### 3.2 Application: Irreversible Investment under Uncertainty
A cornerstone of modern corporate finance is the **real options** approach to investment. A firm has the option to pay a fixed, sunk cost $I$ to build a factory. The factory's value, $P_t$, follows a geometric Brownian motion: $dP = \mu P dt + \sigma P dZ_t$. The firm must choose the optimal time to invest.

The value of the investment option, $F(P)$, must satisfy the HJB equation in the continuation region:
$$ rF = \mu P F'(P) + \frac{1}{2} \sigma^2 P^2 F''(P) $$
This is an ODE with the general solution $F(P) = A_1 P^{\gamma_1} + A_2 P^{\gamma_2}$, where $\gamma_1 > 1$ and $\gamma_2 < 0$ are the roots of the characteristic quadratic. Economic conditions rule out the explosive positive root, so $F(P) = A P^\gamma$. The solution is found by imposing boundary conditions (value matching and smooth pasting) at the optimal investment threshold $P^*$. This yields the solution for the investment threshold:
$$ P^* = \frac{\gamma_1}{\gamma_1 - 1} I $$
where $\gamma_1 = \frac{1}{2} - \frac{\mu}{\sigma^2} + \sqrt{\left(\frac{\mu}{\sigma^2} - \frac{1}{2}\right)^2 + \frac{2r}{\sigma^2}} > 1$. The term $\frac{\gamma_1}{\gamma_1 - 1} > 1$ is the **real options multiplier**. It shows that the firm should wait until the project's value $P$ is significantly greater than the investment cost $I$. This gap represents the **option value of waiting** and increases with uncertainty (volatility $\sigma$).

### 4. Application 2: American Option Pricing
An American put option gives the right to sell an asset at a strike price $K$ at *any* time up to a finite expiration date $T$. This is a finite-horizon optimal stopping problem. We solve it via backward induction on a discretized state space, starting from the known terminal condition at expiration, $V_T(S) = \max(K-S, 0)$.

In [ ]:
sec("Pricing an American Put Option via VFI")
K, R, SIGMA, T_exp = 100, 0.01, 0.2, 1.0
N_periods, N_S = 100, 101 # Time steps and state space grid size
dt = T_exp / N_periods
S_grid = np.linspace(50, 150, N_S)

# Discretize the shock distribution (approximating a random walk)
p_up = 0.5 * (1 + (R - 0.5*SIGMA**2)*np.sqrt(dt)/SIGMA)
p_down = 1 - p_up
u, d = np.exp(SIGMA * np.sqrt(dt)), np.exp(-SIGMA * np.sqrt(dt))

V = np.maximum(K - S_grid, 0) # Value at expiration

for t in range(N_periods - 1, -1, -1):
    # Calculate expected continuation value
    S_up = S_grid * u; S_down = S_grid * d
    V_up = np.interp(S_up, S_grid, V)
    V_down = np.interp(S_down, S_grid, V)
    EV = p_up * V_up + p_down * V_down
    
    # Bellman update
    V = np.maximum(K - S_grid, np.exp(-R * dt) * EV)

V_option = V
exercise_boundary = S_grid[np.isclose(V_option, K - S_grid, atol=1e-3)]
exercise_price = exercise_boundary[0] if len(exercise_boundary)>0 else np.nan
note(f"The optimal exercise boundary is at S* = ${exercise_price:.2f}")

plt.figure(figsize=(10, 7))
plt.plot(S_grid, V_option, label='American Put Value (t=0)')
plt.plot(S_grid, np.maximum(K - S_grid, 0), 'k--', label='Intrinsic Value')
plt.axvline(exercise_price, color='r', ls='--', label=f'Exercise Boundary S*')
plt.title('American Put Option Value and Exercise Boundary')
plt.xlabel('Stock Price (S)'); plt.ylabel('Option Value'); plt.legend()
plt.show()

### 5. Chapter Summary
- **Structure:** Optimal stopping problems involve a binary choice at each period: stop and receive a terminal payoff, or continue and receive a flow payoff plus the discounted expected value of facing the problem again.
- **Bellman Equation:** The value function is the maximum of the stopping value and the continuation value. This structure is solvable with value function iteration.
- **Reservation Value:** The optimal policy is often a simple threshold rule: stop if the state variable crosses a critical **reservation value** (or **optimal stopping boundary**).
- **Continuous Time:** As $\Delta t \to 0$, the Bellman equation converges to the **HJB equation**, a PDE that governs the value function in the continuation region.
- **Real Options:** The theory of irreversible investment under uncertainty is a key application. It shows that volatility creates an **option value of waiting**, causing firms to invest only when the project's value significantly exceeds its cost.

### 6. Exercises

1.  **Secretary Problem Simulation:** Write a simulation to verify the optimality of the $N/e$ rule for the Secretary Problem. For $N=100$, simulate the process 10,000 times for different rejection thresholds `r`. Plot the success probability as a function of `r`. Does the maximum occur near $r=100/e$?

2.  **Comparative Statics (Benefits):** How does the reservation wage in the McCall model change if unemployment benefits `c` increase from 25 to 35? Re-solve for the new reservation wage. Does the average unemployment duration increase or decrease? Explain the economic intuition.

3.  **Comparative Statics (Volatility):** For the real options investment model, plot the optimal investment threshold $P^*$ as a function of volatility $\sigma$. How does increased uncertainty affect the option value of waiting and the firm's willingness to invest? 

4.  **On-the-Job Search:** Extend the McCall model to include on-the-job search. An employed worker with wage `w` gets a new wage offer `w'` with some probability `λ` each period. The Bellman equation for an *employed* worker is now: $V(w) = w + \beta [ (1-\lambda)V(w) + \lambda E[\max(V(w'), V(w))] ]$. Solve this more complex problem using VFI. How does the possibility of on-the-job search affect the initial reservation wage of an unemployed worker?

5.  **American Call Option:** An American *call* option gives the right to *buy* the asset at strike price K. Its stopping value is $\max(S-K, 0)$. It is famously never optimal to exercise an American call option on a non-dividend-paying stock before expiration. Can you modify the VFI code for the American put to price a call option and verify this result? (Hint: The value of the American call should be identical to the value of a European call).